In [6]:
Plot = require 'itorch.Plot'

# Load the data
We first load the weird t7 converted files containing the data. The conversion step is in the <a href="data_loading.ipynb">data loading notebook</a>.

In [1]:
file = torch.DiskFile('dat/facies_vectors.t7', 'r')
facies = file:readObject()
file:close()
file = torch.DiskFile('dat/validation_data_nofacies.t7', 'r')
validate = file:readObject()
file:close()

# Clean the data
### Extract the useful feature vectors
Let's pick out the well logs and the "geologic constraining variables" by simply dropping the facies and the depth data.

In [2]:
print("facies size: ", facies:size()[1], "x", facies:size()[2])
print("validate size: ", validate:size()[1], "x", validate:size()[2])

facies size: 	4149	x	9	
validate size: 	830	x	8	


I've decided to map the wells into a third dimension of a tensor, so that the dims will go as 1=log, 2=depth, 3=well. I'm doing this as a table since the well lengths are variable.

In [3]:
-- initialize
training_data = {}
testing_data = {}
depth = {}

-- build the training wells into the table
training_data["shrimplin"] = facies[{{1,471},{3,9}}]
training_data["alexander"] = facies[{{472,937},{3,9}}]
training_data["shankle"] = facies[{{938,1386},{3,9}}]
training_data["luke"] = facies[{{1387,1847},{3,9}}]
training_data["kimzey"] = facies[{{1848,2286},{3,9}}]
training_data["cross"] = facies[{{2287,2787},{3,9}}]
training_data["nolan"] = facies[{{2788,3202},{3,9}}]
training_data["recruit"] = facies[{{3203,3282},{3,9}}]
training_data["newby"] = facies[{{3283,3745},{3,9}}]
training_data["churchman"] = facies[{{3746,4149},{3,9}}]

-- build the testing wells into the table
testing_data["stuart"] = validate[{{1,474},{2,8}}]
testing_data["crawford"] = validate[{{475,830},{2,8}}]

-- build a depth log for plotting
depth["shrimplin"] = facies[{{1,471},{2}}]
depth["alexander"] = facies[{{472,937},{2}}]
depth["shankle"] = facies[{{938,1386},{2}}]
depth["luke"] = facies[{{1387,1847},{2}}]
depth["kimzey"] = facies[{{1848,2286},{2}}]
depth["cross"] = facies[{{2287,2787},{2}}]
depth["nolan"] = facies[{{2788,3202},{2}}]
depth["recruit"] = facies[{{3203,3282},{2}}]
depth["newby"] = facies[{{3283,3745},{2}}]
depth["churchman"] = facies[{{3746,4149},{2}}]
depth["stuart"] = validate[{{1,474},{1}}]
depth["crawford"] = validate[{{475,830},{1}}]

-- QC
print(training_data)
print(testing_data)
print(depth)

{
  nolan : DoubleTensor - size: 415x7
  luke : DoubleTensor - size: 461x7
  shrimplin : DoubleTensor - size: 471x7
  kimzey : DoubleTensor - size: 439x7
  cross : DoubleTensor - size: 501x7
  newby : DoubleTensor - size: 463x7
  churchman : DoubleTensor - size: 404x7
  shankle : DoubleTensor - size: 449x7
  alexander : DoubleTensor - size: 466x7
  recruit : DoubleTensor - size: 80x7
}
{
  stuart : DoubleTensor - size: 474x7
  crawford : DoubleTensor - size: 356x7
}
{
  nolan : DoubleTensor - size: 415x1
  shankle : DoubleTensor - size: 449x1
  luke : DoubleTensor - size: 461x1
  stuart : DoubleTensor - size: 474x1
  shrimplin : DoubleTensor - size: 471x1
  kimzey : DoubleTensor - size: 439x1
  cross : DoubleTensor - size: 501x1
  newby : DoubleTensor - size: 463x1
  churchman : DoubleTensor - size: 404x1
  crawford : DoubleTensor - size: 356x1
  alexander : DoubleTensor - size: 466x1
  recruit : DoubleTensor - size: 80x1
}


### Normalize the data
As per the literature and Brandon's suggestion, we now normalize the data to have zero mean and unit variance.

In [4]:
mean = {}
stdv = {}

for key,value in pairs(training_data) do --over each well
    mean[key] = torch.Tensor(7)
    stdv[key] = torch.Tensor(7)
    print('Well: ', key)
    for i = 1, 7 do --over each log
        mean[key][i] = training_data[key][{{},{i}}]:mean()
        print('Log ' .. i .. ', Mean: ' .. mean[key][i])
        training_data[key][{{},{i}}]:add(-mean[key][i])
        
        stdv[key][i] = training_data[key][{{},{i}}]:std()
        print('Log ' .. i .. ', Standard Deviation: ' .. stdv[key][i])
        training_data[key][{{},{i}}]:div(stdv[key][i])
    end
    print("\n")
end

Well: 	nolan	
Log 1, Mean: 68.693939759036	
Log 1, Standard Deviation: 32.730215642043	
Log 2, Mean: 0.5924	
Log 2, Standard Deviation: 0.20888527235023	
Log 3, Mean: 3.1340698795181	
Log 3, Standard Deviation: 2.4304927280085	
Log 4, Mean: 12.197361445783	
Log 4, Standard Deviation: 4.8411695655865	
Log 5, Mean: 3.8579469879518	
Log 5, Standard Deviation: 0.87767775335189	
Log 6, Mean: 1.5277108433735	
Log 6, Standard Deviation: 0.49983409155943	
Log 7, Mean: 0.54900722891566	
Log 7, Standard Deviation: 0.28556345258955	

	
Well: 	luke	
Log 1, Mean: 64.777223427332	
Log 1, Standard Deviation: 27.427174260686	
Log 2, Mean: 0.63968980477223	
Log 2, Standard Deviation: 0.22662107159636	
Log 3, Mean: 4.2184381778742	
Log 3, Standard Deviation: 4.8162210685864	
Log 4, Mean: 12.953904555315	
Log 4, Standard Deviation: 6.3295181442742	
Log 5, Mean: 3.660704989154	
Log 5, Standard Deviation: 0.72724200247972	
Log 6, Mean: 1.4663774403471	
Log 6, Standard Deviation: 0.49941019631722	
Log 7, Me


	
Well: 	cross	
Log 1, Mean: 72.699285429142	
Log 1, Standard Deviation: 21.062929262768	
Log 2, Mean: 0.56882884838323	
Log 2, Standard Deviation: 0.28967861069263	
Log 3, Mean: 2.4666666666667	
Log 3, Standard Deviation: 6.536863672027	
Log 4, Mean: 15.574850299401	
Log 4, Standard Deviation: 9.6794269576016	
Log 5, Mean: 3.3258522954092	
Log 5, Standard Deviation: 0.67679195188752	
Log 6, Mean: 1.2974051896208	
Log 6, Standard Deviation: 0.4575732220017	
Log 7, Mean: 0.50618163672655	
Log 7, Standard Deviation: 0.2892806266277	

	
Well: 	newby	
Log 1, Mean: 60.733045356371	
Log 1, Standard Deviation: 33.620632052063	
Log 2, Mean: 0.6748120950324	
Log 2, Standard Deviation: 0.17501719552048	
Log 3, Mean: 3.477969762419	
Log 3, Standard Deviation: 4.9763151781545	
Log 4, Mean: 11.625377969762	
Log 4, Standard Deviation: 5.1300372370773	
Log 5, Mean: 3.7717062634989	
Log 5, Standard Deviation: 0.52500124108663	
Log 6, Mean: 1.6155507559395	
Log 6, Standard Deviation: 0.48699101424912	

Log 7, Standard Deviation: 0.28904042667	

	
Well: 	recruit	
Log 1, Mean: 43.3550625	
Log 1, Standard Deviation: 30.181913663458	
Log 2, Mean: 0.60065	
Log 2, Standard Deviation: 0.32385381527919	
Log 3, Mean: 2.205925	
Log 3, Standard Deviation: 2.7526427107487	
Log 4, Mean: 12.70010625	
Log 4, Standard Deviation: 4.2035419744512	
Log 5, Mean: 15004.491625	
Log 5, Standard Deviation: 35930.104540863	
Log 6, Mean: 2	
Log 6, Standard Deviation: 0	
Log 7, Mean: 0.462425	
Log 7, Standard Deviation: 0.23717924509492	

	


In [5]:
mean = {}
stdv = {}

for key,value in pairs(testing_data) do --over each well
    mean[key] = torch.Tensor(7)
    stdv[key] = torch.Tensor(7)
    print('Well: ', key)
    for i = 1, 7 do --over each log
        mean[key][i] = testing_data[key][{{},{i}}]:mean()
        print('Log ' .. i .. ', Mean: ' .. mean[key][i])
        testing_data[key][{{},{i}}]:add(-mean[key][i])
        
        stdv[key][i] = testing_data[key][{{},{i}}]:std()
        print('Log ' .. i .. ', Standard Deviation: ' .. stdv[key][i])
        testing_data[key][{{},{i}}]:div(stdv[key][i])
    end
    print("\n")
end

Well: 	stuart	
Log 1, Mean: 56.819900843882	
Log 1, Standard Deviation: 28.600303331762	
Log 2, Mean: 0.71246624472574	
Log 2, Standard Deviation: 0.21809118076801	
Log 3, Mean: 3.2905063291139	
Log 3, Standard Deviation: 4.0506328371708	
Log 4, Mean: 11.373945147679	
Log 4, Standard Deviation: 4.8097374633476	
Log 5, Mean: 3.7441476793249	
Log 5, Standard Deviation: 0.58620027327333	
Log 6, Mean: 1.6308016877637	
Log 6, Standard Deviation: 0.48309759705951	
Log 7, Mean: 0.543	
Log 7, Standard Deviation: 0.27979586336928	

	
Well: 	crawford	
Log 1, Mean: 58.666019662921	
Log 1, Standard Deviation: 26.033590787314	
Log 2, Mean: 0.6048595505618	
Log 2, Standard Deviation: 0.3523933340864	
Log 3, Mean: 2.2680617977528	
Log 3, Standard Deviation: 2.2819218565829	
Log 4, Mean: 12.029859550562	
Log 4, Standard Deviation: 5.6424975847849	
Log 5, Mean: 3.5343876404494	
Log 5, Standard Deviation: 0.70918906775178	
Log 6, Mean: 1.7415730337079	
Log 6, Standard Deviation: 0.43838602546969	
Log 7,

In [7]:
plot = Plot():line(training_data[{{1,471},{1}}]:reshape(471), depth[{{1,471}}],'red','gamma ray'):draw()

plot:line(training_data[{{1,471},{2}}]:reshape(471)+2, depth[{{1,471}}],'blue','ILD')
plot:line(training_data[{{1,471},{3}}]:reshape(471)+4, depth[{{1,471}}],'green','dPhi')
plot:line(training_data[{{1,471},{4}}]:reshape(471)+6, depth[{{1,471}}],'brown','NeuPor')
plot:line(training_data[{{1,471},{5}}]:reshape(471)+8, depth[{{1,471}}],'grey','PE')
plot:line(training_data[{{1,471},{6}}]:reshape(471)+10, depth[{{1,471}}],'black','NM_M')
plot:line(training_data[{{1,471},{7}}]:reshape(471)+12, depth[{{1,471}}],'yellow','RelPos')

plot:legend(true)
plot:title("Shrimplin Logs")
plot:redraw()

In [23]:
plot = Plot():line(training_data["shrimplin"][{{},{1}}]:reshape(training_data["shrimplin"]:size(1)), depth["shrimplin"]:reshape(training_data["shrimplin"]:size(1)),'red','gamma'):draw()

plot:line(training_data["shrimplin"][{{},{2}}]:reshape(training_data["shrimplin"]:size(1))+2, depth["shrimplin"]:reshape(training_data["shrimplin"]:size(1)),'blue','ILD')
plot:line(training_data["shrimplin"][{{},{3}}]:reshape(training_data["shrimplin"]:size(1))+4, depth["shrimplin"]:reshape(training_data["shrimplin"]:size(1)),'green','dPhi')
plot:line(training_data["shrimplin"][{{},{4}}]:reshape(training_data["shrimplin"]:size(1))+6, depth["shrimplin"]:reshape(training_data["shrimplin"]:size(1)),'brown','nPor')
plot:line(training_data["shrimplin"][{{},{5}}]:reshape(training_data["shrimplin"]:size(1))+8, depth["shrimplin"]:reshape(training_data["shrimplin"]:size(1)),'grey','PE')
plot:line(training_data["shrimplin"][{{},{6}}]:reshape(training_data["shrimplin"]:size(1))+10, depth["shrimplin"]:reshape(training_data["shrimplin"]:size(1)),'black','NM_M')
plot:line(training_data["shrimplin"][{{},{7}}]:reshape(training_data["shrimplin"]:size(1))+12, depth["shrimplin"]:reshape(training_data["shrimplin"]:size(1)),'yellow','RelPos')

plot:legend(true)
plot:title("Shrimplin Logs")
plot:redraw()

### Extract blind well
Next we separate the Newby well for blind testing.